<a href="https://colab.research.google.com/github/ManviNarang01/AgroInsight/blob/main/AgroInsightCombinedFeature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This model performs a comprehensive analysis of weather and crop data for agricultural decision support. The first part focuses on rainfall prediction using an XGBoost regression model. The weather dataset is preprocessed, and missing rainfall values are filled with zeros. The data is split into training and testing sets, and a preprocessing pipeline is defined to handle numeric features. An XGBoost regressor is then employed to predict rainfall, and the model's performance is evaluated using metrics such as Mean Squared Error (MSE) and R-squared. The second part of the code addresses crop recommendation using a Random Forest classifier. The crop dataset is loaded, and the target variable is label-encoded for classification. Similar to the rainfall prediction, the dataset is split, and a Random Forest model is trained to recommend crops. The accuracy of the crop recommendation model is evaluated, and the code concludes by predicting the recommended crop based on a new set of weather data, incorporating the predicted rainfall from the first model.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline

In [ ]:
# Loading weather dataset
weather_data = pd.read_csv('weather.csv')
weather_data = weather_data.drop(columns=['time'])

<ipython-input-7-fe0c82a996de>:2: DtypeWarning: Columns (1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  weather_data = pd.read_csv('weather.csv')


In [ ]:
# Separating features as (X_rain) and target variable as (y_rain)
X_rain = weather_data.drop(columns=['rain (mm)'])
y_rain = weather_data['rain (mm)']
y_rain = y_rain.fillna(0)

In [ ]:
# Splitting the data for rainfall prediction
X_rain_train, X_rain_test, y_rain_train, y_rain_test = train_test_split(X_rain, y_rain, test_size=0.2, random_state=42)

In [ ]:
# Further preprocessing for rainfall prediction
numeric_features_rain = X_rain.select_dtypes(include=['float64']).columns
numeric_transformer_rain = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler()
)

preprocessor_rain = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_rain, numeric_features_rain)
    ])

In [ ]:
# Creating the rainfall prediction model pipeline
model_xgb_rain = make_pipeline(preprocessor_rain, XGBRegressor(random_state=42))

In [ ]:
# Training the rainfall prediction model
model_xgb_rain.fit(X_rain_train, y_rain_train)

# Making predictions on the rainfall test set
y_pred_rain = model_xgb_rain.predict(X_rain_test)

In [ ]:
# Evaluating the rainfall prediction model using MSE
mse_rain = mean_squared_error(y_rain_test, y_pred_rain)

print(f'Mean Squared Error (Rainfall Prediction): {mse_rain}')

Mean Squared Error (Rainfall Prediction): 0.3772330390036125


In [ ]:
# Loading the crop recommendation dataset
crop_data = pd.read_csv('Crop_recommendation.csv')

# Separating features as(X_crop) and target variable as (y_crop)
X_crop = crop_data.drop(columns=['label'])
y_crop = crop_data['label']


In [ ]:
# Label encoding the target variable for crop recommendation
le_crop = LabelEncoder()
y_crop = le_crop.fit_transform(y_crop)

In [ ]:
# Splitting the data for crop recommendation
X_crop_train, X_crop_test, y_crop_train, y_crop_test = train_test_split(X_crop, y_crop, test_size=0.2, random_state=42)

In [ ]:
# Creating the crop recommendation model pipeline
model_crop = make_pipeline(RandomForestClassifier(n_estimators=100, random_state=42))

In [ ]:
# Training the crop recommendation model
model_crop.fit(X_crop_train, y_crop_train)

# Making predictions on the crop recommendation test set
y_pred_crop = model_crop.predict(X_crop_test)

In [ ]:
# Evaluating the crop recommendation model for Accuracy and classification report
accuracy_crop = accuracy_score(y_crop_test, y_pred_crop)
print(f'Accuracy (Crop Recommendation): {accuracy_crop}')
print('Classification Report (Crop Recommendation):')
print(classification_report(y_crop_test, y_pred_crop))

Accuracy (Crop Recommendation): 0.9931818181818182
Classification Report (Crop Recommendation):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        23
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00        20
           3       1.00      1.00      1.00        26
           4       1.00      1.00      1.00        27
           5       1.00      1.00      1.00        17
           6       1.00      1.00      1.00        17
           7       1.00      1.00      1.00        14
           8       0.92      1.00      0.96        23
           9       1.00      1.00      1.00        20
          10       0.92      1.00      0.96        11
          11       1.00      1.00      1.00        21
          12       1.00      1.00      1.00        19
          13       1.00      0.96      0.98        24
          14       1.00      1.00      1.00        19
          15       1.00      1.00      

In [ ]:
# Making predictions for new weather data
new_weather_data = pd.DataFrame({
    'N': [0],
    'P': [0],
    'K': [0],
    'temperature': [0],
    'humidity': [0],
    'ph': [0],
    'rainfall': [y_pred_rain[-1]]  # Using the predicted rainfall for the last month
})

# Using the trained crop recommendation model to predict the recommended crop
predicted_crop = le_crop.inverse_transform(model_crop.predict(new_weather_data))
print(f'Recommended Crop: {predicted_crop[0]}')

Recommended Crop: kidneybeans
